In [1]:
import requests

In [2]:
from bs4 import BeautifulSoup

In [3]:
import sqlite3
import pandas as pd

##выкачка данных

###первое задание

In [4]:
response_olymp = requests.get('https://www.olympedia.org/sports')

In [5]:
#проверяем, что все нормально работает
print(response_olymp)

<Response [200]>


In [6]:
#смотрим raw
print(response_olymp.text)

<!DOCTYPE html>
<html>
<head>
<title>Olympedia – Sports &amp; disciplines</title>
<meta name="csrf-param" content="authenticity_token" />
<meta name="csrf-token" content="1Y9-8lqBm5Y-jOM1jDd5_Ix3ODuJTkQKxbLOJJrOjzQTr8yKbbuujTFpBjs2SOl6Hzes-fey4QVKhdWZ7LT55A" />
<meta content="text/html; charset=UTF-8" http-equiv="Content-Type">
<meta content="EN" http-equiv="content-language">
<meta content="width=device-width, initial-scale=1" name="viewport">
<link rel="stylesheet" href="/assets/bootstrap.min-03423cd68c2bff228a73520e849f02d78d1236de443661f3f5dcd658dd9cbc46.css" media="all" />
<link rel="stylesheet" href="/assets/bootstrap-sortable-515ee97c8b372ddae7796a36ec518e0785e0e3d3d266162ba2c44a3623540f7b.css" media="all" />
<link rel="stylesheet" href="/assets/jquery-ui-1.11.4.min-35aebb4aa1818460c44d32da4f18101d34f332741a347654907911af0d145e88.css" media="all" />
<link rel="stylesheet" href="/assets/lightbox-3afc0e08d652072ea73df78af7d454f122da80b9124d782fb609a7b4b05e0197.css" media="screen" 

###работаем с конкретными запросами

In [7]:
soup = BeautifulSoup(response_olymp.text, 'html.parser')

rows = soup.find_all('tr')
olymp_sports = []
for row in rows:
    code = row.find('td')
    olymp_sports.append(f'{str(code)[4:7]}')


In [8]:
def get_sports_data(sport):
    url = f'https://www.olympedia.org/sports/{sport}'
    response_discipline = requests.get(url)
    soup_sport = BeautifulSoup(response_discipline.text, 'html.parser')
    discipline_title = (str(soup_sport.find_all('title'))[20:-9])
    rows_sport = soup_sport.find_all('tr')
    for block in rows_sport[1:5]:
        data = block.find('th')
        if 'Participants' in data:
            participants = block.find('td')
        elif 'NOCs' in data:
            nocs = block.find('td')
    description = soup_sport.find('div', class_='description')
    if description:
        description = description.text.strip()
    else:
        description = 'None'
    return sport, discipline_title, participants.text.strip(), nocs.text.strip(), description

In [9]:
print(get_sports_data('KEN')) #пример вывода

('KEN', 'Kendo', '0', '0', 'The Japanese form of fencing, called kendō, was one of the three traditional Japanese sports demonstrated at the 1964 Olympics (along with kyudō and sumō). The fights were not competitive in nature and did not yield a final ranking.')


здесь нужно будет вставить описание почему оно именно так работает и/или красивую таблицу

In [10]:
codes = []
titles = []
num_participants = []
num_nocs = []
descriptions = []

for sport in olymp_sports:
    print(get_sports_data(sport))
    codes.append(get_sports_data(sport)[0])
    titles.append(get_sports_data(sport)[1])
    num_participants.append(get_sports_data(sport)[2])
    num_nocs.append(get_sports_data(sport)[3])
    descriptions.append(get_sports_data(sport)[4])

('IH3', '3-on-3 Ice Hockey', '0', '0', 'Three-on-three ice hockey is a smaller version of the traditional sport. It has never been on the Olympic Programme, but it was an event contested at the 2020 Youth Olympic Winter Games.')
('BK3', '3x3 Basketball', '116', '19', '3x3 basketball is a playground version of the traditional sport of basketball. Before being added to the Olympic Programme, the discipline was contested at the 2010, 2014, and 2018 Youth Olympic Games. 3x3 basketball made its Olympic début at the 2020 Tokyo Olympic Games, where Latvia won the men’s competition, and the United States won the women’s tournament. In the discipline’s second Olympic appearance at Paris 2024, the Dutch men grabbed the Olympic title, just as the German women’s team did.\nAs of 2024, after two Olympic presences in the programme, the United States and the ROC are the only NOCs with two medals. However, no athlete has yet won more than one Olympic medal in 3x3 basketball.\n3x3 basketball is governe

In [11]:
data_sports = {
    'discipline': titles,
    'participants': num_participants,
    'NOCs': num_nocs,
    'description': descriptions
}

data_codes = {'code': codes, 'discipline': titles}

df_codes = pd.DataFrame(data_codes)

df_sports = pd.DataFrame(data_sports)

In [12]:
df_sports.head()

,discipline,participants,NOCs,description
0,3-on-3 Ice Hockey,0,0,Three-on-three ice hockey is a smaller version...
1,3x3 Basketball,116,19,3x3 basketball is a playground version of the ...
2,Acrobatic Gymnastics,0,0,Acrobatics began as a specific sport in the So...
3,Aeronautics,1,1,Aeronautics is governed internationally by the...
4,Alpine Skiing,3036,111,Alpine ski racing is the newer form of ski rac...


In [13]:
df_codes.head()

,code,discipline
0,IH3,3-on-3 Ice Hockey
1,BK3,3x3 Basketball
2,GAC,Acrobatic Gymnastics
3,AER,Aeronautics
4,ALP,Alpine Skiing


###работа с женщинами

In [14]:
def find_sportswomen(num_page):
    fem_url = f"https://www.olympedia.org/athletes/search_athlete?affiliations=&bio=&birthcountry=&birthplace=&competition_type_id=1&continent=&country=&date%{num_page}Bbirthday%{num_page}D=&date%{num_page}Bbirthmonth%{num_page}D=&date%{num_page}Bbirthyear%{num_page}D=&date%{num_page}Bdeathday%{num_page}D=&date%{num_page}Bdeathmonth%{num_page}D=&date%{num_page}Bdeathyear%{num_page}D=&deathcountry=&deathplace=&event_name=&gender=female&given_name=&id=&image=&medal=Any&page={num_page}&season=winter&second_country=&sport=&sport_group=&surname=&year2=&year="
    response_women = requests.get(fem_url)
    women_soup = BeautifulSoup(response_women.text, 'html.parser')
    women_soup_2 = women_soup.find_all('tr')
    sportswomen = []
    for block in women_soup_2:
        athlete_data = block.find_all('td')
        if len(athlete_data) >= 6:
                  id = athlete_data[0].text.strip()
                  name = athlete_data[1].text.strip()
                  discipline = athlete_data[4].text.strip()
                  years = athlete_data[5].text.strip()
                  sportswomen.append((id, name, discipline, years))
    return sportswomen

In [15]:
ids = []
names = []
sports_codes = []
year_begins = []
year_ends = []
for page in range(1, 8):
    print(find_sportswomen(page)) #чтобы было видно, что там вообще получается
    for women in find_sportswomen(page):
        ids.append(int(women[0]))
        names.append(women[1])
        sports_codes.append(women[2])
        if '—' in women[3]:
            year_begins.append(int(women[3].split('—')[0]))
            year_ends.append(int(women[3].split('—')[1]))
        else:
            year_begins.append(int(women[3]))
            year_ends.append(int(women[3]))

[('110371', 'Yekaterina Abramova', 'SSK', '2006—2010'), ('80508', 'Rita Achkina', 'CCS', '1964—1968'), ('137690', 'Carlijn Achtereekte', 'SSK', '2018—2022'), ('127779', 'Vicki Adams', 'CUR', '2014—2018'), ('110248', 'Meghan Agosta', 'IHO', '2006—2018'), ('100432', 'Annica Åhlén', 'IHO', '1998—2002'), ('148378', 'Emma Aicher', 'ALP', '2022'), ('99555', 'Albina Akhatova', 'BTH', '1998—2006'), ('138179', 'Emmy Alasalmi', 'IHO', '2012—2018'), ('80516', 'Brigitte Albrecht-Loretan', 'CCS', '1992—2002'), ('80517', 'Sylvia Albrecht', 'SSK', '1980'), ('80519', 'Tenley Albright', 'FSK', '1952—1980'), ('136767', 'Dzinara Alimbekava', 'BTH', '2018—2022'), ('80522', 'Yuliya Allagulova', 'STK', '1992'), ('100433', 'Lotta Almblad', 'IHO', '1998—2002'), ('138143', 'Estelle Alphand', 'ALP', '2012—2018'), ('127860', 'Katharina Althaus', 'SJP', '2012—2022'), ('128503', 'Livia Altmann', 'IHO', '2014—2018'), ('96192', 'Jeannette Altwegg', 'FSK', '1948—1952'), ('96193', 'Ingrīda Amantova', 'LUG', '1980—1984

In [16]:
print(names)

['Yekaterina Abramova', 'Rita Achkina', 'Carlijn Achtereekte', 'Vicki Adams', 'Meghan Agosta', 'Annica Åhlén', 'Emma Aicher', 'Albina Akhatova', 'Emmy Alasalmi', 'Brigitte Albrecht-Loretan', 'Sylvia Albrecht', 'Tenley Albright', 'Dzinara Alimbekava', 'Yuliya Allagulova', 'Lotta Almblad', 'Estelle Alphand', 'Katharina Althaus', 'Livia Altmann', 'Jeannette Altwegg', 'Ingrīda Amantova', 'Erin Ambrose', 'Zinaida Amosova', 'An Sang-Mi', 'Jamie Anderson', 'Anna Andersson', 'Cecilia Andersson', 'Ebba Andersson', 'Gunilla Andersson', 'Lina Andersson', 'Carola Anding', 'Gunn Margit Andreassen', 'Marina Anissina', 'Daniela Anschütz-Thoms', 'Dana Antal', 'Déborah Anthonioz', 'Jakara Anthony', 'Katrin Apel', 'Gillian Apps', 'Brooke Apshkrum', 'Shizuka Arakawa', 'Debbie Armstrong', 'Emily Arthur', 'Mao Asada', 'Jeanne Ashworth', 'Jenni Asserholt', 'Izzy Atkin', 'Susan Auch', 'Inger Aufles', 'Tatyana Averina-Barabash', 'Irina Avvakumova', 'Stien Baas-Kaiser', 'Lina Bäcklin', 'Josie Baff', 'Shannon B

In [25]:
women_data = {
    'id': ids,
    'name': names,
    'career_start': year_begins,
    'career_end': year_ends
}

df_sportswomen = pd.DataFrame(women_data)
df_women_disciplines = pd.DataFrame({'woman_id': ids, 'sport': sports_codes})

df_sportswomen.head()

,id,name,career_start,career_end
0,110371,Yekaterina Abramova,2006,2010
1,80508,Rita Achkina,1964,1968
2,137690,Carlijn Achtereekte,2018,2022
3,127779,Vicki Adams,2014,2018
4,110248,Meghan Agosta,2006,2018


In [32]:
df_women_disciplines.head()

,woman_id,sport
0,110371,SSK
1,80508,CCS
2,137690,SSK
3,127779,CUR
4,110248,IHO


In [51]:
con = sqlite3.connect("olympics.db")
cur = con.cursor()

In [52]:
con.execute("""
CREATE TABLE woman (
    id INTEGER PRIMARY KEY,
    name TEXT,
    career_start INTEGER,
    career_end INTEGER
)
""")

df_sportswomen.to_sql(name='woman', con=con, if_exists='append', index=False)

1400

In [53]:
con.execute("DROP TABLE IF EXISTS sport_codes")
con.execute("""
CREATE TABLE sport_codes (
    "index" INTEGER PRIMARY KEY,
    code TEXT,
    discipline TEXT
)
""")

df_codes.to_sql(name='sport_codes', con=con, if_exists='append', index=False)

132

In [41]:
#con.execute("""
#CREATE TABLE IF NOT EXISTS woman_sport (
#    woman_id INTEGER,
#    sport TEXT,
#    PRIMARY KEY (woman_id)
#    FOREIGN KEY (woman_id) REFERENCES woman(id)
#)
#""")

#df_women_disciplines.to_sql(name='woman_sport', con=con, if_exists='append', index=False)

In [56]:
cur.execute("""
CREATE TABLE sports(
    sport_id INT,
    discipline INT,
    participants INT,
    nocs INT,
    description TEXT,
    PRIMARY KEY (sport_id),
    FOREIGN KEY (sport_id) REFERENCES sport_codes("index")
)
""")

df_sports.to_sql(name='sports', con=con, if_exists='append', index=False)

132

In [57]:
cur.execute("""
SELECT *
FROM woman
WHERE career_start >= 2000
LIMIT 50
""")